# Rabi oscillations measurement

In [1]:
from collections import OrderedDict
import datetime
import numpy as np
import time
import os

try: 
    pulsedmasterlogic
except NameError:
    manager.startModule('logic', 'pulsedmasterlogic')
try: 
    scannerlogic
except NameError:
    manager.startModule('logic', 'scannerlogic')
try: 
    optimizer
except NameError:
    manager.startModule('logic', 'optimizerlogic')
try:
    sequencegenerator
except NameError:
    manager.startModule('logic', 'sequencegenerator')
try:
    pulsedmeasurement
except NameError:
    manager.startModule('gui', 'pulsedmeasurement')  

In [2]:
def write_to_logfile(nametag, timestamp, name, **kwargs):
    """ Write parameters to custom logfile with name nametag """
    if type(timestamp) is not str:
        timestamp = str(timestamp)
    parameters = list(kwargs)
    if len(parameters) == 1 and type(kwargs[parameters[0]]) is OrderedDict:
        param_dict = kwargs[parameters[0]]
        parameters = list(param_dict)
        kwargs = param_dict
    log_dir = pulsedmeasurement._save_logic.get_path_for_module('CustomLogfiles')
    log_path = os.path.join(log_dir, nametag + '.txt')
    if not os.path.isfile(log_path):
        with open(log_path, 'w') as logfile:
            logfile.write('# timestamp\t')
            logfile.write('# Name\t')
            for param in parameters:
                logfile.write(param + '\t')
            logfile.write('\n#\n')
    with open(log_path, 'a') as logfile:
        logfile.write(timestamp + '\t')
        logfile.write(name + '\t')
        for param in parameters:
            logfile.write('{0:3.6e}\t'.format(kwargs[param]))
        logfile.write('\n')
    return

In [11]:
mw_freq = 1608.89e+6 # in MHz
mw_power = -9

# Set new values
pulsedmasterlogic.set_ext_microwave_settings(power=mw_power, frequency=mw_freq-100.0e+6, use_ext_microwave=True)
time.sleep(0.5)

In [12]:
# Get the dictionary containing the default parameters for the desired sequence and alter the values as needed.
# generate_params = pulsedmasterlogic.generate_method_params['rabi']
generate_params=OrderedDict()
generate_params['name'] = 'rabi'
generate_params['microwave_amplitude'] = 0.25
generate_params['number_of_taus'] = 100
generate_params['tau_start'] = 10.0e-9
generate_params['tau_step'] = 10.0e-9

# get the current settings with the following property:
for setting, value in pulsedmasterlogic.ext_microwave_settings.items():
    print('{0}: {1}'.format(setting, value))

# Call the actual generation method using a wrapper method and pass the parameter set
pulsedmasterlogic.generate_predefined_sequence('rabi', generate_params)
# Give it a moment to generate
time.sleep(0.5)

# The created object is a PulseBlockEnsemble instance, a recipe to create a waveform.
# You can get all created PulseBlockEnsembles using this property (keys are ensemble names, values are the object instances):
#for ensemble_name in pulsedmasterlogic.saved_pulse_block_ensembles.keys():
#    print(ensemble_name)

# The "with_load" flag can be used to also load the waveform into the channels after creating it
pulsedmasterlogic.sample_ensemble('rabi', with_load=True)
# Wait for the sampling to finish. There is a status dictionary that can be used to check for process status.
while pulsedmasterlogic.status_dict['sampload_busy']:
    time.sleep(0.1)
    
pulsedmasterlogic.set_measurement_settings(invoke_settings=False, 
                                           controlled_variable= generate_params['tau_start'] + 
                                           np.arange(generate_params['number_of_taus'])*generate_params['tau_step'],
                                           number_of_lasers=generate_params['number_of_taus'], 
                                           laser_ignore_list=[], 
                                           alternating=False, 
                                           units=('s', 'arb. u.'))
time.sleep(0.5)

pulsedmasterlogic.set_fast_counter_settings(record_length=4.0e-6, number_of_gates=generate_params['number_of_taus'])
time.sleep(0.5)

# You can get a read-only view of the current measurement settings by using this property:
# for setting, value in pulsedmasterlogic.measurement_settings.items():
#     print('{0}:\n  {1}'.format(setting, value))
import winsound
frequency = 300  # Set Frequency To 2500 Hertz
duration = 1000  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)
# You can get a read-only view of the current measurement settings by using this property:
# for setting, value in pulsedmasterlogic.measurement_settings.items():
#     print('{0}:\n  {1}'.format(setting, value))

power: -9.0
frequency: 1508890000.0
use_ext_microwave: True


In [13]:
pulsedmasterlogic.save_measurement_data(tag='Rabi_'+str(generate_params['microwave_amplitude']), with_error=True)

In [14]:
sequencegenerator._generation_parameters['rabi_period'] = 194.1e-9

# Start the measurement

In [4]:
# If everything is properly set, we can start a measurement simply by calling:
pulsedmasterlogic.toggle_pulsed_measurement(True)
# Wait until the pulsedmeasurementlogic is actually busy and the measurement is running
while pulsedmeasurementlogic.module_state() != 'locked':
    time.sleep(0.2)

# Stop the measurement

In [5]:
pulsedmasterlogic.toggle_pulsed_measurement(False)
# Wait until the pulsedmeasurementlogic is actually idle and the measurement is stopped
while pulsedmeasurementlogic.module_state() == 'locked':
    time.sleep(0.2)

In [33]:
rabi['measurement_time'] = 60
rabi['refocus_interval'] = 2*60

In [31]:
setup['poi'] = poimanagerlogic.active_poi.get_key()
refocus_poi(setup['poi'])

2D gaussian fit not successfull
118340.0 0.0


True

In [34]:
do_rabi()

(False, 5.2306788674027365e-07, 28.522823671285114, 1911797.7328563172, 0.81606813291014346)

In [35]:
pulsedmasterlogic.save_measurement_data('s','Rabi_'+setup['NV_name']+'_'+str(rabi['mw_amp'])+'V',True, False)

'G:\\\\data\\\\QudiData\\2018\\07\\20180711\\PulsedMeasurement'

In [6]:
def do_rabi(generate_new=True, save_tag='', load_tag=''):
    
    length = rabi['number_of_taus'] * rabi['tau_step']
            
    # sanity check for long sequences:
    if length > 3e-3:
        print("Rabi exceeds 3 ms too long for AWG5014C")
        return True,0,0,0,0
    # generate ensemble object
#     rabi['tau_res'] = rabi['length'] / (rabi['points'] - 1)
    tau_arr = np.arange(rabi['number_of_taus']) * rabi['tau_step']+rabi['tau_start']
    if generate_new:
        sequencegeneratorlogic.delete_ensemble('Rabi')
        sequencegeneratorlogic.generate_rabi('Rabi', rabi['tau_start'], rabi['tau_step'], rabi['number_of_taus'], rabi['mw_freq'], 
                                        rabi['mw_amp'], 'a_ch1', setup['laser_length'] , 1.0, setup['aom_delay'], 
                                        setup['wait_length'], setup['seq_trig'], setup['gate'])
        if 'Rabi' not in sequencegeneratorlogic.saved_pulse_block_ensembles:
            sequencegeneratorlogic.log.error('Ensemble generation of Rabi in notebook timed out.')
            return
        pulsedmasterlogic.sample_block_ensemble('Rabi', True)
    else:
        pulsedmasterlogic.load_asset_into_channels('Rabi')
    while pulsedmasterlogic.status_dict['sauplo_ensemble_busy'] or pulsedmasterlogic.status_dict['loading_busy']:
        time.sleep(0.2)
    pulsedmasterlogic.do_fit('No Fit')
    # set parameters in analysis tab
    pulsedmasterlogic.measurement_sequence_settings_changed(tau_arr, rabi['number_of_taus'], 100e-6, [], False)
    pulsedmasterlogic.fast_counter_settings_changed(setup['fc_binwidth'], setup['laser_length']+1e-6, rabi['number_of_taus'])
    pulsedmasterlogic.analysis_interval_changed(1)
    pulsedmeasurementlogic.measurement_tag = save_tag
    # perform measurement
    pulsedmasterlogic.start_measurement(load_tag)
    while not pulsedmasterlogic.status_dict['measurement_running']:
        time.sleep(0.2)
    user_terminated = False
    start_time = time.time()
    while time.time() - start_time < rabi['measurement_time']:
        if not pulsedmasterlogic.status_dict['measurement_running']:
            user_terminated = True
            break
        time.sleep(0.5)
    pulsedmasterlogic.manually_pull_data() 
    time.sleep(1)
    pulsedmasterlogic.stop_measurement(save_tag)
    while pulsedmasterlogic.status_dict['measurement_running']:
        time.sleep(0.2)
        time.sleep(2)
    # do fit
    x, y, param_dict = pulsedmeasurementlogic.do_fit('Rabi')
    new_freq = param_dict.params['frequency'].value
    contrast = 2.*param_dict.params['amplitude'].value
    offset = param_dict.params['offset'].value
    period = 1./new_freq
    
#     # save data
#     if save_tag is None:
#         pulsedmasterlogic.save_measurement_data('s','Rabi_'+setup['NV_name']+'_'+str(rabi['mw_amp'])+'V',True)
#     else:
#         pulsedmasterlogic.save_measurement_data('s','Rabi_'+setup['NV_name']+'_'+str(rabi['mw_amp'])+'V',True)
    
    # write logfile
    write_to_logfile('Rabi_log', str(datetime.datetime.now()),name= setup['NV_name'], amplitude=rabi['mw_amp'], period=period,
                     frequency=new_freq, contrast=contrast, offset= offset)
    return user_terminated, period, contrast*100, new_freq, offset

In [9]:
def refocus_poi(poi=None, max_drift=0.5, min_counts=setup["min_counts"]):
    if poi==None:
        return False
    scannerlogic.stop_scanning()
    poimanagerlogic.go_to_poi(poi)
    pos_start = scannerlogic.get_position()
    setup['NV_name']=poimanagerlogic.poi_list[poi]._name
    poimanagerlogic.optimise_poi(poi)
    while optimizerlogic.getState() == 'idle':
        time.sleep(0.2)
    while optimizerlogic.getState() != 'idle':
        time.sleep(0.2)
    time.sleep(5)
    pos_end=scannerlogic.get_position()
    print(counterlogic.countdata.mean(),abs(pos_end[0]-pos_start[0]))
    if (abs(pos_end[0]-pos_start[0])>max_drift or abs(pos_end[1]-pos_start[1])>max_drift or 
       abs(pos_end[2]-pos_start[2])>max_drift or counterlogic.countdata.mean()<min_counts):
        scannerlogic.set_position('',x=pos_start[0],y=pos_start[1],z=pos_start[2])
        poimanagerlogic.set_new_position(poi,pos_start)
        print(abs(pos_end[0]-pos_start[0])>max_drift or abs(pos_end[1]-pos_start[1])>max_drift or 
       abs(pos_end[2]-pos_start[2])>max_drift or counterlogic.countdata.mean()<min_counts)
        return False
        write_to_logfile('PositionRefocus_log', str(datetime.datetime.now()),name= setup['NV_name'], 
                     x_pos=pos_start[0], y_pos=pos_start[1], z_pos=pos_start[2],sucess=0)
    write_to_logfile('PositionRefocus_log', str(datetime.datetime.now()),name= setup['NV_name'], 
                     x_pos=pos_end[0], y_pos=pos_end[1], z_pos=pos_end[2],sucess=1)
    return True
def do_rabi_refocus(poi):
    end_measure=False
    #refocus_poi(poi)
    rabi_total_time = rabi['measurement_time']
    rabi_runtime = 0.0
    rabi['measurement_time'] = rabi['refocus_interval']
    end_measure = do_rabi(True, 'rabi_refocus','')[0]
    rabi_runtime += rabi['refocus_interval']
    while rabi_total_time > rabi_runtime:
        for i in range(1):
            end_measure = not refocus_poi(poi)
            if not end_measure:
                break
        if end_measure:
            break
        end_measure = do_rabi(False, 'rabi_refocus', 'rabi_refocus')[0]
        rabi_runtime += rabi['refocus_interval']
        print(rabi_runtime)
        if end_measure:
            break
    rabi['measurement_time'] = rabi_total_time
    write_to_logfile('rabi_log', str(datetime.datetime.now()),name= setup['NV_name'],runtime=rabi_runtime)
    time.sleep(2)
    return end_measure